In [5]:
import cvxpy as cp
import numpy as np

np.random.seed(10)
(m, n) = (30, 10)

A = np.random.rand(m, n)
A = np.asmatrix(A)

b = np.random.rand(m, 1)
b = np.asmatrix(b)

c_nom = np.ones((n, 1)) + np.random.rand(n, 1)
c_nom = np.asmatrix(c_nom)

In [19]:
#given 
#each c_i do not deviate more than 25% of nominal c
lhs1 = (0.25*1 - 1)*c_nom
rhs1 = (0.25*1 + 1)*c_nom

#average of C do not deviate more than 10% of nominal values
lhs2 = (1*0.1 - 1)*sum(c_nom)/n
rhs2 = (1*0.1 + 1)*sum(c_nom)/n

# constraint formulation
g = np.r_[rhs1, lhs1, rhs2, lhs2]
F = np.r_[np.eye(n), -np.eye(n), np.ones((1, n))/n, -np.ones((1, n))/n]

lang_multiplier = cp.Variable(g.size)
x = cp.Variable(n)

S = cp.reshape(A*x,(30,1))

# cost function
def totalCost(a, b):
  return cp.matmul(a.T, b)

# Applying dual on given problem (from c part):
dual_constraints = [S>=b, cp.matmul(F.T, lang_multiplier)==x, lang_multiplier>=0]
dual_objective = cp.Minimize(totalCost(lang_multiplier, g))
dual_prob = cp.Problem(dual_objective, dual_constraints)
dual_prob.solve()

# since optimal value of dual is also optimal value of primal
x_robust = x.value

constraint = [S>=b]
objective = cp.Minimize(totalCost(c_nom, x))
prob = cp.Problem(objective, constraint)
prob.solve()
x_nominal = x.value
x_nominal = cp.reshape(x_nominal,(10,1))
x_robust = cp.reshape(x_robust,(10,1))

print("Cost of X Nominal:",(c_nom.T*x_nominal).value )
print("Cost of X Robust:",(c_nom.T*x_robust).value )

# primal problem
c = cp.Variable(n)
S1 = cp.reshape(F*c,(22,1))
constraint = [S1<=g]

nominal_objective = cp.Maximize(totalCost(c, x_nominal))
robust_objective = cp.Maximize(totalCost(c, x_robust))

nominal_func_problem = cp.Problem(nominal_objective, constraint)
nominal_func = nominal_func_problem.solve()

robust_func_problem = cp.Problem(robust_objective, constraint)
robust_func = robust_func_problem.solve()

print("\nWorst case for nominal x:",nominal_func)
print("Worst case for robust x:",robust_func)

Cost of X Nominal: [[2.10927146]]
Cost of X Robust: [[2.52320886]]

Worst case for nominal x: 7.2215622012361225
Worst case for robust x: 3.165961051233025
